<h1>Segmenting and Clustering Troronto Neighborhoods</h1>

This Project is part of the IBM Data Science Professional Certificate Caoptone.

<b>Author</b>: Julian Oellrich

<h2 id="libraries">1. Setup and Libraries</h2>

Import Libraries

In [79]:
# Comuptation Libraries
import pandas as pd
import numpy as np

# API Communication
import json
import requests

# Plotting 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# Maps and geocoding
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium

# ML Algorithms
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# json & API
import json, lxml
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# data scrping modules
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Print Statement
print('Libraries imported!')

Libraries imported!


<h2 id="data_import">2. Data Import</h2>

<h3>Scrape Data from Wikipedia</h3>

Scraping the data from wikipedia page with BeautifulSoup Librariy

In [80]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

Extracting data from table on Wikipedia page

In [81]:
table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody

Write Table data into new dataframe

In [82]:
columns = ['PostalCode', 'Borough', 'Neighbourhood']
data = dict({key:[]*len(columns) for key in columns})

for row in table.find_all('tr'):
    for i,column in zip(row.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

df = pd.DataFrame.from_dict(data=data)[columns]
print(df.shape)
df.head(10)

(180, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


<h3>Clean Data</h3>

<h4>Remove not assigned Boroughs</h4>

Only the cells that have an assigned borough should be processed. That means cells with a borough that is 'Not assigned' should be removed from the data frame.

1. The first step is to count how many rows have a 'Not assigned' Borough:

In [83]:
print_statement = 'There are {} rows where Borough is Not assigned'.format(
    df[df['Borough'] == 'Not assigned'].shape[0])
print(print_statement)

There are 77 rows where Borough is Not assigned


2. Drop the rows where Borough is 'Not assigned' and write it into new dataframe df_cleaned

In [84]:
df_cleaned = df.drop(df[df['Borough'] == 'Not assigned'].index) 
df_cleaned.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


3. Check if there are any rows with Not assigned Borough in the new dataframe

In [85]:
print_statement = 'There are {} rows where Borough is Not assigned'.format(
    df_cleaned[df['Borough'] == 'Not assigned'].shape[0])
print(print_statement)

There are 0 rows where Borough is Not assigned


<h4>Fill in not assigned neighborhood</h4>

If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough

Let's check how many rows have a 'Not assigned' Neighbourhood:

In [86]:
print_statement = 'There are {} rows where Neighbourhood is Not assigned'.format(
    df_cleaned[df_cleaned['Neighbourhood'] == 'Not assigned'].shape[0])
print(print_statement)

There are 0 rows where Neighbourhood is Not assigned


<b>Conclusion:</b> It shows, that there are <b> no rows</b> with 'Not assigned' Neighbourhood but assigned Borough

<h4>Check duplicate Postalcodes</h4>

In [87]:
# create new test dataframe with only neighbourhoods and postal codes
df_neigh = df_cleaned[['PostalCode','Neighbourhood']]

# check Postalcode duplicates and create new bool column that marks duplicates with True
df_neigh['duplicate bool'] = df_neigh['PostalCode'].duplicated(keep = 'first')

# Output value count of duplicate PostalCode values
df_neigh['duplicate bool'].value_counts()

False    103
Name: duplicate bool, dtype: int64

<b>Conclusion:</b> There are no duplicate PostCode values 

<h4>Final cleaned DataFrame</h4>

Finally change the columnname 'PostalCode' to 'Postal Code'

In [88]:
df_cleaned.rename(columns={'PostalCode': 'Postal Code'}, inplace = True)

Write cleaned data to Dataframe df_toronto and print shape

In [89]:
print_statement = '\n The toronto dataframe has {} rows and {} columns \n'.format(df_cleaned.shape[0], df_cleaned.shape[1])
print(print_statement)
df_cleaned.head()


 The toronto dataframe has 103 rows and 3 columns 



,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2 id="geodata">4. Geo-Data of Toronto Boroughs</h2>

<h3>Import geospatial coordinates</h3>

Import latitude and longitude data from csv file 'Toronto_Geospatial_Coordinates.csv' provided by Coursera

In [90]:
geospatial_toronto = pd.read_csv('Toronto_Geospatial_Coordinates.csv')
geospatial_toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3>Merge geospatial coordinates to toronto DataFrame</h3>

Merging 'geospatial_toronto' to 'df_cleaned' with key 'Postal Code' and save it to new DataFrame 'df_toronto'

In [95]:
df_toronto = pd.merge(df_cleaned, geospatial_toronto, how= 'inner', on ='Postal Code')
df_toronto.head(8)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188


<h3>Create a Toronto Map</h3>

<h4>Get Toronto Coordinatesp</h4>

In [96]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, lng))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h4>Create a map of New York with neighborhoods superimposed on top.</h4>

In [100]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[lat, lng], zoom_start=10)

# add markers to map
for lat, lng, postalcode, borough in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Postal Code'], df_toronto['Borough']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto